In [1]:
import cv2
import pandas as pd
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# /kaggle/input/25-indian-bird-species-with-226k-images/training_set/training_set içindeki klasörleri listele
os.listdir('/kaggle/input/25-indian-bird-species-with-226k-images/training_set/training_set')

['Indian Roller',
 'White-Breasted Waterhen',
 'Indian Grey Hornbill',
 'Common Myna',
 'Rufous Treepie',
 'Cattle Egret',
 'House Crow',
 'Ruddy Shelduck',
 'Gray Wagtail',
 'Forest Wagtail',
 'Common Tailorbird',
 'Sarus Crane',
 'White Wagtail',
 'Jungle Babbler',
 'Hoopoe',
 'Coppersmith Barbet',
 'Common Kingfisher',
 'Brown-Headed Barbet',
 'Indian Peacock',
 'Asian Green Bee-Eater',
 'Indian Pitta',
 'Red-Wattled Lapwing',
 'Northern Lapwing',
 'White-Breasted Kingfisher',
 'Common Rosefinch']

In [3]:
data_dir='/kaggle/input/25-indian-bird-species-with-226k-images/training_set/training_set'
img_width,img_height=224,224

idg = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data_gen=idg.flow_from_directory(directory=data_dir,
                                  target_size=(img_width, img_height),
                                  class_mode='categorical',
                                  subset='training')

test_data_gen=idg.flow_from_directory(directory=data_dir,
                                  target_size=(img_width, img_height),
                                  class_mode='categorical',
                                  subset='validation')

Found 18106 images belonging to 25 classes.
Found 4514 images belonging to 25 classes.


## Transfer Learning with VGG16

In [4]:
base_model = VGG16(weights='imagenet', input_shape=(img_width, img_height, 3), include_top=False)

model = Sequential()
model.add(base_model)  # transfer ettik
for layer in base_model.layers:  # freeze ettik
    layer.trainable = False  # Layerleri eğitmiyoruz, donduruyoruz

model.add(Flatten())
model.add(Dense(25, activation='softmax'))  # 25 sınıf için çıkış katmanı

model.compile(optimizer='adam', 
              loss='categorical_crossentropy',  # Çok sınıflı sınıflandırma için
              metrics=['accuracy', 'top_k_categorical_accuracy'])  # Ek metrikler eklendi

model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [5]:
history=model.fit(train_data_gen, validation_data=test_data_gen, epochs=15, verbose=1)

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1725970883.152556     117 service.cc:145] XLA service 0x7bef8c00d180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725970883.152620     117 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1725970883.152626     117 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  1/566 ━━━━━━━━━━━━━━━━━━━━ 3:03:02 19s/step - accuracy: 0.0312 - loss: 3.7497 - top_k_categorical_accuracy: 0.0938

I0000 00:00:1725970895.509878     117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


417/566 ━━━━━━━━━━━━━━━━━━━━ 1:46 717ms/step - accuracy: 0.3849 - loss: 2.3768 - top_k_categorical_accuracy: 0.6816

KeyboardInterrupt: 

# Result: Transfer learning with VGG16 showed a 10-fold increase already in the second epoch